In [1]:
import pandas as pd
import numpy as np

f = pd.read_csv('f.csv', encoding = 'utf-8').reset_index(drop=True)
g = pd.read_csv('g.csv', encoding = 'utf-8').reset_index(drop=True)

f.topic_idx = f.topic_idx.astype('int8')
g.topic_idx = g.topic_idx.astype('int8')

train_sentences = f['title']
valid_sentences = g['title']
data = f['title']
train_labels = f['topic_idx']
valid_labels = g['topic_idx']

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

tokenizer.fit_on_texts(data.astype(str))
print('총 단어 갯수: ', len(tokenizer.word_index))

# 2회 이상만 vocab_size에 포함
def get_vocab_size(threshold):
  cnt = 0
  for x in tokenizer.word_counts.values():
    if x >= threshold:
      cnt = cnt + 1
  return cnt
vocab_size = get_vocab_size(2)
print('vocab_size: ', vocab_size)

oov_tok = '<OOV>' # 사전에 없는 단어
vocab_size = get_vocab_size(2)

tokenizer = Tokenizer(oov_token=oov_tok, num_words=vocab_size)
tokenizer.fit_on_texts(data.astype(str))
#print(tokenizer.word_index)
print('단어 사전 개수: ', len(tokenizer.word_counts))

print(train_sentences[:2])
train_sequences = tokenizer.texts_to_sequences(train_sentences.astype(str))
valid_sequences = tokenizer.texts_to_sequences(valid_sentences.astype(str))
print(train_sequences[:2])

# 문장의 최대 길이
max_length = max(len(x) for x in train_sequences)
print('문장 최대 길이:', max_length)

# 문장 길이를 동일하게 맞춘다
trunc_type = 'post'
padding_type = 'post'
train_padded = pad_sequences(train_sequences, 
                             truncating=trunc_type, 
                             padding=padding_type, 
                             maxlen=17)
valid_padded = pad_sequences(valid_sequences, 
                             truncating=trunc_type, 
                             padding=padding_type, 
                             maxlen=17)
train_labels = np.asarray(train_labels)
valid_labels = np.asarray(valid_labels)
print('샘플:', train_padded[:1])

총 단어 갯수:  100155
vocab_size:  67158
단어 사전 개수:  100155
0    LG 유플러스 LTE 라우터 U 와이파이 쏙 출시
1    삼성 증권 메 리츠 증권 유상증자 주가 영향 제한
Name: title, dtype: object
[[105, 3149, 3115, 43794, 1188, 6556, 4953, 78], [33, 1064, 2600, 6232, 1064, 11075, 1266, 514, 742]]
문장 최대 길이: 40
샘플: [[  105  3149  3115 43794  1188  6556  4953    78     0     0     0     0
      0     0     0     0     0]]


In [26]:
f['topic_idx']

0         4
1         2
2         1
3         4
4         5
         ..
140430    0
140431    0
140432    0
140433    0
140434    0
Name: topic_idx, Length: 140435, dtype: int8

In [27]:
g['topic_idx']

0        1
1        1
2        3
3        2
4        5
        ..
13663    3
13664    1
13665    6
13666    6
13667    1
Name: topic_idx, Length: 13668, dtype: int8

In [2]:
f.topic_idx.value_counts()

2    151972
6    151281
4    129084
1    119466
3    110215
5     66835
0     30625
Name: topic_idx, dtype: int64

In [29]:
y0 = np.where(f.topic_idx == 0, 1, 0)
y00 = np.where(g.topic_idx == 0, 1, 0)

In [36]:
for i in range(0, 7):
  y0 = np.where(f.topic_idx == i, 1, 0)
  y00 = np.where(g.topic_idx == i, 1, 0)

  model_a = Sequential()
  model_a.add(Embedding(22417, 128, input_length = 17))
  model_a.add(Bidirectional(LSTM(128, return_sequences =True)))
  model_a.add(tf.keras.layers.Dropout(0.2))
  model_a.add(Dense(32, activation = 'tanh'))
  model_a.add(Dense(8, activation = 'tanh'))
  model_a.add(Dense(1, activation = 'sigmoid'))
  model_a.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  history_a = model_a.fit(train_padded, y0, epochs=1, batch_size=BATCH_SIZE,
                    validation_data = (valid_padded, y00), shuffle = True)
  print(history_a.history['val_loss'])
  print(history_a.history['val_acc'])

InvalidArgumentError:  indices[30,0] = 26720 is not in [0, 22417)
	 [[node sequential_9/embedding_9/embedding_lookup (defined at <ipython-input-36-1b305f60dce5>:15) ]] [Op:__inference_train_function_266830]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_9/embedding_9/embedding_lookup:
 sequential_9/embedding_9/embedding_lookup/264442 (defined at C:\Users\admin\Anaconda3\lib\contextlib.py:112)

Function call stack:
train_function


In [34]:
history0 = model_a.fit(train_padded, y0, epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_data = (valid_padded, y00), 
                    callbacks=[checkpoint, early_stop],
                    shuffle = True)

Epoch 1/20


InvalidArgumentError:  indices[26,4] = 23313 is not in [0, 22417)
	 [[node sequential_7/embedding_7/embedding_lookup (defined at <ipython-input-34-0512cba9b10d>:4) ]] [Op:__inference_train_function_254546]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_7/embedding_7/embedding_lookup:
 sequential_7/embedding_7/embedding_lookup/252158 (defined at C:\Users\admin\Anaconda3\lib\contextlib.py:112)

Function call stack:
train_function


In [7]:
# 모델 구축
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional, SimpleRNN

# callback : 가장 좋은 loss의 가중치 저장
checkpoint_path = 'best_performed_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                save_weights_only=True,
                                                save_best_only=True,
                                                monitor='val_loss',
                                                verbose=1)

# 얼리스탑 // 3회로
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# 라벨링 원핫
Y = pd.get_dummies(train_labels).values
print('Shape of train_label tensor:', Y.shape)
Y_val = pd.get_dummies(valid_labels).values
print('Shape of valid_label tensor:', Y_val.shape)

# 러닝레이트 0.001 이 default인데, 적은 에포크에서도 과적합 발생하니 줄여보자
opt = tf.keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')


model = Sequential()
model.add(Embedding(67158, 128, input_length=17))
model.add(tf.keras.layers.SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(32, activation = 'tanh'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


EPOCHS = 20
BATCH_SIZE = 32

model.summary()

Shape of train_label tensor: (759478, 7)
Shape of valid_label tensor: (13679, 7)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 17, 128)           8596224   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 17, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 231       
Total params: 8,732,167
Trainable param

In [8]:
history = model.fit(train_padded, Y, epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_data = (valid_padded, Y_val), 
                    callbacks=[checkpoint, early_stop],
                    shuffle = True)

Epoch 1/20
 3442/23734 [===>..........................] - ETA: 36:09 - loss: 1.0302 - accuracy: 0.6382

KeyboardInterrupt: 

In [16]:
## epoch 6 정도가 과적합 막고 베스트로 보임, 새로이 모델링

model1 = Sequential()
model1.add(Embedding(31547, 128, input_length=17))
model1.add(tf.keras.layers.SpatialDropout1D(0.2))
model1.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(32, activation = 'relu'))
model1.add(tf.keras.layers.Dropout(0.2))
model1.add(Dense(7, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

EPOCHS = 6
BATCH_SIZE = 32

model1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 33, 128)           6377472   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 33, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 231       
Total params: 6,513,415
Trainable params: 6,513,415
Non-trainable params: 0
____________________________________________

In [17]:
# train/valid set 합쳐서 4회 학습

X = np.concatenate((train_padded, valid_padded))
y = np.concatenate((Y, Y_val))

history1 = model1.fit(X, y, epochs=6, batch_size=32,
                    shuffle = True)

Epoch 1/6
4816/4816 [==============================] - 642s 133ms/step - loss: 1.3232 - accuracy: 0.4848
Epoch 2/6
4816/4816 [==============================] - 621s 129ms/step - loss: 0.8653 - accuracy: 0.7137
Epoch 3/6
4816/4816 [==============================] - 601s 125ms/step - loss: 0.7390 - accuracy: 0.7635
Epoch 4/6
4816/4816 [==============================] - 606s 126ms/step - loss: 0.6716 - accuracy: 0.7875
Epoch 5/6
4816/4816 [==============================] - 603s 125ms/step - loss: 0.6169 - accuracy: 0.8045
Epoch 6/6
4816/4816 [==============================] - 605s 126ms/step - loss: 0.5770 - accuracy: 0.8173


In [18]:
# submission 예측

from konlpy.tag import Okt

submit = pd.read_csv('test_data.csv', encoding = 'utf-8', index_col = 0)

submit.title = submit.title.str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ一-龥]', ' ')
submit.reset_index(drop = True, inplace = True)
okt=Okt()
h = pd.DataFrame()
for i in range(len(submit.title)):
    c = okt.pos(submit.title[i])
    d = [y for (y, x) in c if (x=='Noun')|(x == 'Alpha')|(x=='Foreign')]

    e = ''
    for j in range(len(d)):
        if j < (len(d) - 1):
            e += d[j]+' '
        else:
            e += d[j]
    h = pd.concat([h, pd.DataFrame(np.array([e]).reshape(1,1))])
h.columns = ['title']
h.reset_index(drop=True, inplace=True)

submit_sentences = h['title']
submit_X = tokenizer.texts_to_sequences(submit_sentences)

submit_padded = pad_sequences(submit_X, 
                              truncating=trunc_type, 
                              padding=padding_type, 
                              maxlen=17)


result = model1.predict(submit_padded, batch_size = BATCH_SIZE)

submission = []

for i in range(len(result)):
    submission.append(np.argmax(result[i]))

In [19]:
# 결과 저장
subm = pd.read_csv('sample_submission.csv')
subm.topic_idx = submission
subm.to_csv('submission0902.csv', index = None)

In [17]:
## 크롤링 제외, 원 트레이닝 세트만 가지고

import pandas as pd
import numpy as np

f = pd.read_csv('f1.csv', encoding = 'utf-8').reset_index(drop=True)
g = pd.read_csv('g1.csv', encoding = 'utf-8').reset_index(drop=True)

f.topic_idx = f.topic_idx.astype('int8')
g.topic_idx = g.topic_idx.astype('int8')

train_sentences = f['title']
valid_sentences = g['title']
data = f['title']
train_labels = f['topic_idx']
valid_labels = g['topic_idx']

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

tokenizer.fit_on_texts(data)
print('총 단어 갯수: ', len(tokenizer.word_index))

# 2회 이상만 vocab_size에 포함
def get_vocab_size(threshold):
  cnt = 0
  for x in tokenizer.word_counts.values():
    if x >= threshold:
      cnt = cnt + 1
  return cnt
vocab_size = get_vocab_size(2)
print('vocab_size: ', vocab_size)

oov_tok = '<OOV>' # 사전에 없는 단어
vocab_size = get_vocab_size(2)

tokenizer = Tokenizer(oov_token=oov_tok, num_words=vocab_size)
tokenizer.fit_on_texts(data)
#print(tokenizer.word_index)
print('단어 사전 개수: ', len(tokenizer.word_counts))

print(train_sentences[:2])
train_sequences = tokenizer.texts_to_sequences(train_sentences)
valid_sequences = tokenizer.texts_to_sequences(valid_sentences)
print(train_sequences[:2])

# 문장의 최대 길이
max_length = max(len(x) for x in train_sequences)
print('문장 최대 길이:', max_length)

# 문장 길이를 동일하게 맞춘다
trunc_type = 'post'
padding_type = 'post'
train_padded = pad_sequences(train_sequences, 
                             truncating=trunc_type, 
                             padding=padding_type, 
                             maxlen=max_length)
valid_padded = pad_sequences(valid_sequences, 
                             truncating=trunc_type, 
                             padding=padding_type, 
                             maxlen=max_length)
train_labels = np.asarray(train_labels)
valid_labels = np.asarray(valid_labels)
print('샘플:', train_padded[:1])

총 단어 갯수:  22417
vocab_size:  13789
단어 사전 개수:  22417
0        터키 국방 쿠르드 민병대 예정 철수 중
1    학부모 방과후 필요 명 중 명 초등 교실 선호
Name: title, dtype: object
[[80, 852, 840, 3541, 853, 1056, 33], [2382, 5746, 426, 5, 33, 5, 2124, 1565, 2201]]
문장 최대 길이: 17
샘플: [[  80  852  840 3541  853 1056   33    0    0    0    0    0    0    0
     0    0    0]]


In [18]:
# 모델 구축
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional, SimpleRNN

# callback : 가장 좋은 loss의 가중치 저장
checkpoint_path = 'best_performed_model.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                save_weights_only=True,
                                                save_best_only=True,
                                                monitor='val_loss',
                                                verbose=1)

# 얼리스탑 // 3회로
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# 라벨링 원핫
Y = pd.get_dummies(train_labels).values
print('Shape of train_label tensor:', Y.shape)
Y_val = pd.get_dummies(valid_labels).values
print('Shape of valid_label tensor:', Y_val.shape)

# 러닝레이트 0.001 이 default인데, 적은 에포크에서도 과적합 발생하니 줄여보자
opt = tf.keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

model = Sequential()
model.add(Embedding(22417, 128, input_length=17))
model.add(tf.keras.layers.SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

EPOCHS = 20
BATCH_SIZE = 32

model.summary()

Shape of train_label tensor: (31866, 7)
Shape of valid_label tensor: (13668, 7)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 17, 128)           2869376   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 17, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_12 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 7)                 231       
Total params: 3,005,319
Trainable params

In [21]:
history = model.fit(train_padded, Y, epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_data = (valid_padded, Y_val), 
                    callbacks=[checkpoint, early_stop],
                    shuffle = True)

Epoch 1/20
996/996 [==============================] - 80s 77ms/step - loss: 1.3210 - accuracy: 0.4666 - val_loss: 0.7843 - val_accuracy: 0.7256

Epoch 00001: val_loss improved from inf to 0.78432, saving model to best_performed_model.ckpt
Epoch 2/20
996/996 [==============================] - 78s 78ms/step - loss: 0.6886 - accuracy: 0.7702 - val_loss: 0.5904 - val_accuracy: 0.8094

Epoch 00002: val_loss improved from 0.78432 to 0.59038, saving model to best_performed_model.ckpt
Epoch 3/20
996/996 [==============================] - 74s 74ms/step - loss: 0.4981 - accuracy: 0.8469 - val_loss: 0.5646 - val_accuracy: 0.8172

Epoch 00003: val_loss improved from 0.59038 to 0.56457, saving model to best_performed_model.ckpt
Epoch 4/20
996/996 [==============================] - 75s 75ms/step - loss: 0.3985 - accuracy: 0.8786 - val_loss: 0.5464 - val_accuracy: 0.8276

Epoch 00004: val_loss improved from 0.56457 to 0.54638, saving model to best_performed_model.ckpt
Epoch 5/20
996/996 [============

In [22]:
## epoch 4 정도가 과적합 막고 베스트로 보임, 새로이 모델링

model1 = Sequential()
model1.add(Embedding(22417, 128, input_length=17))
model1.add(tf.keras.layers.SpatialDropout1D(0.2))
model1.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(32, activation = 'relu'))
model1.add(tf.keras.layers.Dropout(0.2))
model1.add(Dense(7, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

EPOCHS = 4
BATCH_SIZE = 32

model1.summary()

# train/valid set 합쳐서 4회 학습

X = np.concatenate((train_padded, valid_padded))
y = np.concatenate((Y, Y_val))

history1 = model1.fit(X, y, epochs=4, batch_size=32,
                    shuffle = True)

# submission 예측

from konlpy.tag import Okt

submit = pd.read_csv('test_data.csv', encoding = 'utf-8', index_col = 0)

submit.title = submit.title.str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ一-龥]', ' ')
submit.reset_index(drop = True, inplace = True)
okt=Okt()
h = pd.DataFrame()
for i in range(len(submit.title)):
    c = okt.pos(submit.title[i])
    d = [y for (y, x) in c if (x=='Noun')|(x == 'Alpha')|(x=='Foreign')]

    e = ''
    for j in range(len(d)):
        if j < (len(d) - 1):
            e += d[j]+' '
        else:
            e += d[j]
    h = pd.concat([h, pd.DataFrame(np.array([e]).reshape(1,1))])
h.columns = ['title']
h.reset_index(drop=True, inplace=True)

submit_sentences = h['title']
submit_X = tokenizer.texts_to_sequences(submit_sentences)

submit_padded = pad_sequences(submit_X, 
                              truncating=trunc_type, 
                              padding=padding_type, 
                              maxlen=max_length)


result = model1.predict(submit_padded, batch_size = BATCH_SIZE)

submission = []

for i in range(len(result)):
    submission.append(np.argmax(result[i]))
    
    
# 결과 저장
subm = pd.read_csv('sample_submission.csv')
subm.topic_idx = submission
subm.to_csv('submission0901_onlyTrain.csv', index = None)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 17, 128)           2869376   
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 17, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_14 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 231       
Total params: 3,005,319
Trainable params: 3,005,319
Non-trainable params: 0
____________________________________________